In [26]:
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
from tqdm import tqdm
import os

In [27]:
import numpy as np

data = np.load("data/candidate_dataset.npz")
print(data.files)

['x_train', 'y_train', 'x_val', 'y_val']


In [35]:
data = np.load("data/candidate_dataset.npz")

X_train = data["x_train"]
y_train = data["y_train"]
X_val   = data["x_val"]
y_val   = data["y_val"]

# 🔥 FIX: convert one-hot → class index
if y_train.ndim > 1:
    y_train = np.argmax(y_train, axis=1)

if y_val.ndim > 1:
    y_val = np.argmax(y_val, axis=1)

print("Train:", X_train.shape, y_train.shape)
print("Val:", X_val.shape, y_val.shape)


Train: (7007, 28, 28, 3) (7007,)
Val: (1003, 28, 28, 3) (1003,)


In [36]:
class DermaDataset(Dataset):
    def __init__(self, images, labels):
        self.images = images
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        x = self.images[idx]

        # If image has 3 channels → convert to grayscale
        if x.ndim == 3 and x.shape[-1] == 3:
            x = x.mean(axis=-1)

        x = torch.tensor(x, dtype=torch.float32).unsqueeze(0) / 255.0
        y = torch.tensor(self.labels[idx], dtype=torch.long)

        return x, y

In [37]:
train_ds = DermaDataset(X_train, y_train)
val_ds   = DermaDataset(X_val, y_val)

train_loader = DataLoader(train_ds, batch_size=64, shuffle=True)
val_loader   = DataLoader(val_ds, batch_size=64, shuffle=False)

In [38]:
class CustomCNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.features = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),

            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Dropout(0.25),

            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.AdaptiveAvgPool2d(1)
        )

        self.classifier = nn.Linear(128, 7)

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        return self.classifier(x)

In [39]:
device = "cuda" if torch.cuda.is_available() else "cpu"

model = CustomCNN().to(device)

criterion = nn.CrossEntropyLoss(label_smoothing=0.1)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3)

In [40]:
os.makedirs("model", exist_ok=True)

epochs = 15
train_losses = []
val_accuracies = []
best_acc = 0.0

for epoch in range(epochs):
    model.train()
    running_loss = 0.0

    for x, y in tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}"):
        x, y = x.to(device), y.to(device)

        optimizer.zero_grad()
        outputs = model(x)
        loss = criterion(outputs, y)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    avg_loss = running_loss / len(train_loader)
    train_losses.append(avg_loss)

    model.eval()
    preds_all, y_all = [], []

    with torch.no_grad():
        for x, y in val_loader:
            x = x.to(device)
            preds = model(x).argmax(1).cpu().numpy()
            preds_all.extend(preds)
            y_all.extend(y.numpy())

    acc = accuracy_score(y_all, preds_all)
    val_accuracies.append(acc)

    if acc > best_acc:
        best_acc = acc
        torch.save(model.state_dict(), "model/best_model.pth")

    print(f"Epoch {epoch+1} | Loss: {avg_loss:.4f} | Val Acc: {acc:.4f}")

Epoch 1/15: 100%|██████████| 110/110 [00:35<00:00,  3.12it/s]


KeyboardInterrupt: 

In [ ]:
plt.figure(figsize=(10,4))

plt.subplot(1,2,1)
plt.plot(train_losses)
plt.title("Training Loss")

plt.subplot(1,2,2)
plt.plot(val_accuracies)
plt.title("Validation Accuracy")

plt.show()

In [ ]:
import torch
import numpy as np
import torch.nn as nn

class CustomCNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.features = nn.Sequential(
            nn.Conv2d(1, 32, 3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),

            nn.Conv2d(32, 64, 3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Dropout(0.25),

            nn.Conv2d(64, 128, 3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.AdaptiveAvgPool2d(1)
        )
        self.classifier = nn.Linear(128, 7)

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        return self.classifier(x)

def evaluate(npz_path):
    data = np.load(npz_path)

    X = data["x_val"]
    y = data["y_val"]

    # RGB → Grayscale
    if X.ndim == 4 and X.shape[-1] == 3:
        X = X.mean(axis=-1)

    X = torch.tensor(X, dtype=torch.float32).unsqueeze(1) / 255.0

    model = CustomCNN()
    model.load_state_dict(torch.load("model/best_model.pth", map_location="cpu"))
    model.eval()

    with torch.no_grad():
        preds = model(X).argmax(1).numpy()

    acc = (preds == y).mean()
    print("Accuracy:", acc)